# Supernovae population for the Roman survey
In this notebook, we simulate population of lensed supernovae for the Roman survey and simulate image of a 

random lensed supernovae. It follows following steps:

1. Simulate lensed supernovae population
2. Choose a lens at random
3. Set observation time and other image configuration
4. Simulate image of a selected lens
5. Visualize it

Before running this notebook, please download the download psfs from cached_webb_psf
(https://github.com/LSST-strong-lensing/data_public/webbpsf), where the
psfs have been generated ahead of time so that they can be loaded from a file.

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import slsim
from slsim.Lenses.lens_pop import LensPop
from slsim.Pipelines.roman_speclite import configure_roman_filters
from slsim.Pipelines.roman_speclite import filter_names
import speclite
from astropy.cosmology import FlatLambdaCDM
from astropy.units import Quantity
import slsim.Sources as sources
import slsim.Deflectors as deflectors
import slsim.Pipelines as pipelines
import corner
import time
from slsim.ImageSimulation.image_simulation import point_source_coordinate_properties
from slsim.Plots.plot_functions import create_image_montage_from_image_list
from slsim.ImageSimulation.roman_image_simulation import lens_image_roman

**WARNING**: LOCAL JWST PRD VERSION PRDOPSSOC-068 DOESN'T MATCH THE CURRENT ONLINE VERSION PRDOPSSOC-072
Please consider updating pysiaf, e.g. pip install --upgrade pysiaf or conda update pysiaf


## Set Roman configurations

In [2]:
# Import default Roman Space Telescope configuration
path = os.path.dirname(slsim.__file__)
module_path, _ = os.path.split(path)
skypy_config = os.path.join(module_path, "data/SkyPy/euclid-roman-lsst-like.yml")

# generate Roman filters
configure_roman_filters()

# import filter bands and make them recogniable in speclite
roman_filters = filter_names()
# filters are ['Roman-F062', 'Roman-F087', 'Roman-F106', 'Roman-F129', 'Roman-F158', 'Roman-F184', 'Roman-F146', 'Roman-F213']

speclite.filters.load_filters(
    roman_filters[0],
    roman_filters[1],
    roman_filters[2],
    roman_filters[3],
    roman_filters[4],
    roman_filters[5],
    roman_filters[6],
    roman_filters[7],
)

## Simulate lensed supernovae population

In [3]:
# define a cosmology
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

# define a sky area
deflector_sky_area = Quantity(value=.05, unit="deg2")
source_sky_area = Quantity(value=.10, unit="deg2")
host_sky_area = Quantity(value=.02, unit="deg2")
sky_area = Quantity(value=100, unit="deg2")

# define limits in the intrinsic deflector and source population (in addition to the skypy config file)
# kwargs_deflector_cut = {"band": "F062", "band_max": 22, "z_min": 0.01, "z_max": 1}
# kwargs_source_cut = {"z_min": 0.02, "z_max": 2} 
kwargs_deflector_cut = {"band": "F062", "band_max": 28, "z_min": 0.01, "z_max": 2.5}
kwargs_source_cut = {"z_min": 0.01, "z_max": 5}

time_range = np.linspace(-50, 500, 550)
modeldir = "/pscratch/sd/e/emlin/lensedsupernovae/data_public/sncosmo_sn_models/SALT3.NIR_WAVEEXT"

In [4]:
# generate galaxy population using skypy pipeline.
galaxy_simulation_pipeline = pipelines.SkyPyPipeline(
    skypy_config=skypy_config, sky_area=deflector_sky_area, filters=None, cosmo=cosmo
)

# generate galaxy population using skypy pipeline for host galaxies. These galaxies will
#  be used as host galaxy candidates.
galaxy_simulation_pipeline_host = pipelines.SkyPyPipeline(
    skypy_config=skypy_config, sky_area=host_sky_area, filters=None, cosmo=cosmo
)

# Initiate deflector population class
lens_galaxies = deflectors.EllipticalLensGalaxies(
    galaxy_list=galaxy_simulation_pipeline.red_galaxies,
    kwargs_cut=kwargs_deflector_cut,
    kwargs_mass2light={},
    cosmo=cosmo,
    sky_area=deflector_sky_area,
)

/pscratch/sd/e/emlin/lensedsupernovae/slsim/slsim/Deflectors/DeflectorPopulation/elliptical_lens_galaxies.py:48: UserWarning: Angular size is converted to arcsec because provided input_catalog_type is skypy. If this is not correct, please refer to the documentation of the class you are using
  galaxy_list = param_util.catalog_with_angular_size_in_arcsec(


In [5]:
# generate supernovae population
supernovae_catalog = sources.SupernovaeCatalog.SupernovaeCatalog(
    sn_type="Ia",
    band_list=["F184", "F129", "F062"],
    lightcurve_time=time_range,
    absolute_mag_band="bessellb",
    absolute_mag=None,
    mag_zpsys="ab",
    cosmo=cosmo,
    skypy_config=skypy_config, 
    sky_area=source_sky_area,
    host_galaxy_candidate=galaxy_simulation_pipeline_host.blue_galaxies,
)

# lightcurve=True doesn't work b/c the supernovae_catalog lightcurve is hardcoded to use lsst bands.
supernovae_data = supernovae_catalog.supernovae_catalog(
    host_galaxy=True, lightcurve=False
)

kwargs_sn = {
    "variability_model": "light_curve",
    "kwargs_variability": {"supernovae_lightcurve", "F184", "F129", "F062"},
    "sn_type": "Ia",
    "sn_absolute_mag_band": "bessellb",
    "sn_absolute_zpsys": "ab",
    "lightcurve_time": np.linspace(-50, 100, 150),
    "sn_modeldir": modeldir,
}

source_SNIa = sources.PointPlusExtendedSources(
    point_plus_extended_sources_list=supernovae_data,
    cosmo=cosmo,
    sky_area=source_sky_area,
    kwargs_cut={},
    catalog_type="skypy",
    source_size=None,
    pointsource_type='supernova',
    pointsource_kwargs=kwargs_sn,
    extended_source_type='single_sersic'
)

/pscratch/sd/e/emlin/lensedsupernovae/slsim/slsim/Sources/Supernovae/supernovae_pop.py:95: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  numerator = integrate.quad(
/pscratch/sd/e/emlin/lensedsupernovae/slsim/slsim/Sources/Supernovae/supernovae_pop.py:95: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  numerator = integrate.quad(
/pscratch/sd/e/emlin/lensedsupernovae/slsim/slsim/Sources/Supernovae/supernovae_pop.py:95: IntegrationWarning: The integral is probably

IndexError: list index out of range

In [10]:
supernovae_catalog.supernovae_catalog(
    host_galaxy=True, lightcurve=False
)

/pscratch/sd/e/emlin/lensedsupernovae/slsim/slsim/Sources/Supernovae/supernovae_pop.py:95: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  numerator = integrate.quad(
/pscratch/sd/e/emlin/lensedsupernovae/slsim/slsim/Sources/Supernovae/supernovae_pop.py:95: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  numerator = integrate.quad(
/pscratch/sd/e/emlin/lensedsupernovae/slsim/slsim/Sources/Supernovae/supernovae_pop.py:95: IntegrationWarning: The integral is probably

IndexError: list index out of range

In [ ]:
# Initiate LensPop class
lens_pop = LensPop(
    deflector_population=lens_galaxies,
    source_population=source_SNIa,
    cosmo=cosmo,
    sky_area=sky_area,
)

In [ ]:
start = time.time()

# Draw lens population
kwargs_lens_cut = {"min_image_separation": 0.1, "max_image_separation": 10}
lens_population = lens_pop.draw_population(
    kwargs_lens_cuts=kwargs_lens_cut, speed_factor=10
)

end = time.time()
print(f"Execution time: {end - start:.2f} seconds")

In [ ]:
print("Number of lenses:", len(lens_population))

lens_samples = []
labels = [
    r"$\sigma_v$",
    r"$\log(M_{*})$",
    r"$\theta_E$",
    r"$z_{\rm l}$",
    r"$z_{\rm s}$",
    r"$m_{\rm source}$",
    r"$m_{\rm lens}$",
]

for gg_lens in lens_population:
    vel_disp = gg_lens.deflector_velocity_dispersion()
    m_star = gg_lens.deflector_stellar_mass()
    theta_e = gg_lens.einstein_radius[0]
    zl = gg_lens.deflector_redshift
    zs = gg_lens.source_redshift_list[0]
    source_mag = min(gg_lens.point_source_magnitude(band="F129", lensed=True)[0])
    deflector_mag = gg_lens.deflector_magnitude(band="F129")
    lens_samples.append(
        [vel_disp, np.log10(m_star), theta_e, zl, zs, source_mag, deflector_mag]
    )


In [ ]:
lens_samples = np.array(lens_samples)
## clean the population. It removes supernovae with nan value for magnitude.
cleaned_data = lens_samples[~np.isnan(lens_samples).any(axis=1)]

In [ ]:
hist2dkwargs = {
    "plot_density": False,
    "plot_contours": False,
    "plot_datapoints": True,
    "color": "b",
    "data_kwargs": {"ms": 5},
}
corner.corner(cleaned_data, labels=labels, **hist2dkwargs)
# corner.corner(np.array(data_cleaned), labels=labels, **hist2dkwargs)
plt.show()

## Simulate images 

Notes on image montages:
- noise turned off (both bg and poisson)
- source is so faint that you can only see the deflector when both source and deflector are turned on
- As we expect, (deflector and source) - (deflector only) = source only
- image one seems to reach the peak at day0
- image 2 doesn't seem to show up at all?
    - ex: in lens 3, 14, 35, one of the images reaches max brightness at day 0, then decreases, but the other image stays the same throughout
- light curves don't match what we see in the time series images
    - ex: lens 14: the light curve reaches a peak and then plateaus, but in the time series images, the brightness decreases after day 0 as it should.
- why is there a ring shape around both objects (lens and some sources)?
- light curves don't match 1a supernovae light curves (see lightcurves notebook)


In [ ]:
import random

# Pick 10 random systems from lens_population and display their lightcurves and image montages
random.seed(42)

random_indices = random.sample(range(len(lens_population)), 10)

sample_times = np.array([-50, -25, 0, 50, 75, 
                         100, 125, 150, 175, 200, 
                         225, 250, 275, 300, 325])

def make_time_series(lens_class, with_source, with_deflector):
    """Generate list of images across time for given source/deflector flags."""
    time_series_images = []
    for t in sample_times:
        image_roman = lens_image_roman(
            lens_class=lens_class,
            band="F129",
            mag_zero_point=27,
            num_pix=71,
            transform_pix2angle=np.array([[0.11, 0], [0, 0.11]]),
            detector=1,
            detector_pos=(2000, 2000),
            oversample=5,
            psf_directory="/pscratch/sd/e/emlin/lensedsupernovae/data_public/webbpsf/cached_webb_psf",
            t_obs=t,
            with_source=with_source,
            with_deflector=with_deflector,
            ra=30,
            dec=-30,
            add_noise=False,
            poisson_noise=False,
            seed=None,
        )
        time_series_images.append(image_roman)
    return time_series_images

for idx in random_indices:
    lens_class = lens_population[idx]
    
    # Light Curve
    times = np.linspace(-50, 500, 500)
    magnitude = lens_class.point_source_magnitude(
        band="F062", lensed=True, time=times
    )

    plt.figure(figsize=(6, 4))
    for i in range(len(magnitude[0])):
        plt.plot(times, magnitude[0][i], label=f"Image {i+1}")
    plt.gca().invert_yaxis()
    plt.ylabel(r"$Mag_{F129}$")
    plt.xlabel("Time [days]")
    plt.xlim(-50, 500)
    plt.title(f"Lens system {idx}")
    plt.legend()
    plt.show()

    # Image Montages
    images_both      = make_time_series(lens_class, with_source=True,  with_deflector=True)
    images_deflector = make_time_series(lens_class, with_source=False, with_deflector=True)
    images_source    = make_time_series(lens_class, with_source=True,  with_deflector=False)
    images_sub       = [img_b - img_d for img_b, img_d in zip(images_both, images_deflector)]

    pix_coord = point_source_coordinate_properties(
        lens_class=lens_class,
        band="F184",
        mag_zero_point=27,
        delta_pix=0.11,
        num_pix=71,
        transform_pix2angle=np.array([[0.11, 0], [0, 0.11]]),
    )["image_pix"]

    titles = [
        "1. Source=True, Deflector=True",
        "2. Source=False, Deflector=True",
        "3. Source=True, Deflector=False",
        "4. Source Only (Both - Deflector)"
    ]
    images = [images_both, images_deflector, images_source, images_sub]

    for title, imgs in zip(titles, images):
        fig = create_image_montage_from_image_list(
            num_rows=3,
            num_cols=5,
            images=imgs,
            time=sample_times,
            image_center=pix_coord,
        )
        fig.suptitle(f"Lens {idx} - {title}", fontsize=14, y=1.02)
        fig.subplots_adjust(top=0.92)
        plt.show()